<a href="https://colab.research.google.com/github/ldaniel-hm/eml_tabular/blob/main/Ejemplo_de_Wrappers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cómo usar los Wrapper Básicos de Gymnasyum.


> Luis Daniel Hernández (ldaniel at um.es) - 2025


En **Gymnasium**, los **wrappers** son clases que te permiten modificar el comportamiento de un entorno sin alterar directamente el código fuente de este. Existen distintos tipos de wrappers para:

1. **Observaciones** (`ObservationWrapper`)  
2. **Acciones** (`ActionWrapper`)  
3. **Recompensas** (`RewardWrapper`)

A continuación se explica cómo puedes crear y usar cada uno de ellos.

Como ejemplo final se muestra cómo discretizar tanto las observaiones como las acciones para el entorno Lunar Lander (https://gymnasium.farama.org/environments/box2d/lunar_lander/).

- **Recuerda** que la forma correcta de discritar las observaciones es con mosaicos.

- Consulta https://gymnasium.farama.org/api/wrappers/table/ para más wrappers.

---

## 1. **Observation Wrapper**
Este tipo de wrapper modifica las observaciones que recibe tu agente. Por ejemplo, imagina que quieres normalizar tus observaciones o aplicar algún procesamiento.


In [1]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.core import ObservationWrapper

class MyObservationWrapper(ObservationWrapper):
    def __init__(self, env: Env):
        super().__init__(env)
        # Aquí podrías guardar parámetros para la transformación,
        # o inicializar variables internas si es necesario.

    def observation(self, obs):
        # Esta función recibe la observación original y retorna la modificada.

        # Ejemplo: Escalar observación dividiendo entre 255 (para imagen)
        # return obs / 255.0

        # Si no haces nada especial, devuelves la observación original:
        return obs

# Para usarlo
env = MyObservationWrapper(gym.make("CartPole-v1"))
obs, info = env.reset()

- El método clave es `observation(self, obs)`, que retorna la nueva observación.
- Al usar `MyObservationWrapper`, cada vez que se llame a `env.reset()` o `env.step(...)`, la observación pasará por este método antes de devolverse a tu agente.

---


## 2. **Action Wrapper**
Modifica las **acciones** que tu agente envía al entorno. Se suele usar para, por ejemplo, **discretizar** o **escalar** acciones cuando tu entorno usa acciones continuas.


In [2]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.core import ActionWrapper

class MyActionWrapper(ActionWrapper):
    def __init__(self, env: Env):
        super().__init__(env)
        # Puedes guardar parámetros sobre cómo vas a transformar la acción.

    def action(self, act):
        # Recibe la acción que tu agente emite
        # y retorna la acción modificada para el entorno.

        # Ejemplo: Rescalar acción a otro rango:
        # new_act = act * 2.0
        # return new_act

        return act

    def reverse_action(self, act):
        # (Opcional) Define cómo revertir la acción
        # si tu agente necesita ver la acción "original".
        # Esto es útil si necesitas hacer tracking de las acciones
        # o en entornos donde uses wrappers anidados.
        return act

# Para usarlo
env = MyActionWrapper(gym.make("CartPole-v1"))
obs, info = env.reset()
action = 1  # Ejemplo
next_obs, reward, done, truncated, info = env.step(action)


- El método principal es `action(self, act)`, que define cómo se traduce la acción de tu agente a una acción válida para el entorno.
- `reverse_action(self, act)` se utiliza para revertir la transformación si fuera necesario.

---



## 3. **Reward Wrapper**
Permite modificar la **recompensa** que retorna el entorno después de cada paso. Por ejemplo, podrías **castigar** más fuertemente un suceso indeseado, o **escalar** la recompensa.



In [3]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.core import RewardWrapper

class MyRewardWrapper(RewardWrapper):
    def __init__(self, env: Env):
        super().__init__(env)

    def reward(self, rew):
        # Aquí defines cómo modificar la recompensa original.

        # Ejemplo: multiplicar la recompensa por un factor de 0.5
        # new_rew = rew * 0.5
        # return new_rew

        return rew

# Para usarlo
env = MyRewardWrapper(gym.make("CartPole-v1"))
obs, info = env.reset()
action = 1  # Ejemplo
next_obs, reward, done, truncated, info = env.step(action)




- El método principal es `reward(self, rew)`, que recibe la recompensa original y devuelve la modificada.

---



## **4. Uso de Varios Wrappers a la Vez**
Los wrappers se pueden **encadenar**. Por ejemplo, si quieres usar uno de observación y otro de acción al mismo tiempo, puedes simplemente anidar las llamadas:

```python
env = gym.make("CartPole-v1")
env = MyActionWrapper(env)
env = MyObservationWrapper(env)
env = MyRewardWrapper(env)
```
De esta manera, cada Wrapper se aplicará en el orden en que lo has encadenado.

---



In [8]:
!pip install swig
!pip install "gymnasium[box2d]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379450 sha256=99ea38c25a326dd4ba3b00e02b2a8e908cfc9991ee52e24f9fbb8854d68f6500
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


In [12]:
import gymnasium as gym
import numpy as np
from gymnasium import Env
from gymnasium.core import ObservationWrapper, ActionWrapper
from gymnasium.spaces import MultiDiscrete, Discrete


class DiscretizeObservationWrapper(ObservationWrapper):
    """
    Discretiza las observaciones de un entorno que originalmente son continuas.
    Para LunarLanderContinuous-v2, la observación tiene 8 variables continuas.
    """

    def __init__(self, env: Env, num_bins=10):
        super().__init__(env)
        self.num_bins = num_bins

        # Obtenemos los límites originales del espacio de observaciones (Box).
        # Son arrays de forma (8,). A veces pueden ser muy grandes o incluso inf.
        self.obs_low = np.where(np.isfinite(env.observation_space.low),
                                env.observation_space.low,
                                -1.0)  # Ajuste si hay -inf
        self.obs_high = np.where(np.isfinite(env.observation_space.high),
                                 env.observation_space.high,
                                 1.0)   # Ajuste si hay +inf

        # Creamos un espacio MultiDiscrete con 'num_bins' para cada una
        # de las 8 dimensiones de observación.
        self.observation_space = MultiDiscrete([num_bins]*8)

    def observation(self, obs):
        """
        Convierte el estado continuo (8,) en un array de 8 índices discretos.
        """
        discrete_obs = []

        for i in range(len(obs)):
            value = obs[i]
            low_i = self.obs_low[i]
            high_i = self.obs_high[i]

            # En caso de que el rango sea 0 (podría pasar si low==high en algún dim),
            # evitamos división por cero
            if high_i == low_i:
                # Si el rango es cero, devolvemos un solo bin (por ejemplo 0).
                discrete_obs.append(0)
                continue

            # Ajustamos valor a [low_i, high_i]
            value = np.clip(value, low_i, high_i)

            # Escalamos a [0,1]
            scaled_value = (value - low_i) / (high_i - low_i)

            # mapeamos a [0, num_bins - 1]
            bin_index = int(scaled_value * (self.num_bins - 1))

            discrete_obs.append(bin_index)

        return np.array(discrete_obs, dtype=np.int64)


class DiscretizeActionWrapper(ActionWrapper):
    """
    Discretiza la acción continua del LunarLanderContinuous-v2.
    El entorno original tiene acciones en [-1,1] para cada uno
    de los 2 motores (main engine y side engine).
    """

    def __init__(self, env: Env, num_bins=5):
        super().__init__(env)
        self.num_bins = num_bins

        # Creamos un espacio MultiDiscrete para las 2 dimensiones de acción,
        # con 'num_bins' bins en cada dimensión.
        # De este modo, la acción discreta es un vector [a0, a1] con cada a_i en [0..num_bins-1].
        self.action_space = MultiDiscrete([num_bins]*2)

        # Generamos un array con los valores discretos en [-1, 1].
        self.bins = np.linspace(-1, 1, num_bins)

    def action(self, act):
        """
        Convierte la acción discreta [idx0, idx1] en una acción continua [val0, val1].
        """
        # La acción 'act' es un array [idx0, idx1].
        # Mapeamos idx0 -> valor continuo en la dimensión 0
        # Mapeamos idx1 -> valor continuo en la dimensión 1
        continuous_action = np.array([
            self.bins[act[0]],
            self.bins[act[1]]
        ], dtype=np.float32)

        # El entorno espera una acción de forma (2,) en el rango [-1,1].
        return continuous_action

    def reverse_action(self, act):
        """
        (Opcional) Método que, dado un valor continuo, retorna el índice discreto.
        Aquí no se implementa porque rara vez es necesario.
        """
        raise NotImplementedError


if __name__ == "__main__":
    # Creamos el entorno original de LunarLanderContinuous
    env = gym.make("LunarLanderContinuous-v3")

    # Envolvemos el entorno con nuestros wrappers
    env = DiscretizeObservationWrapper(env, num_bins=10)
    env = DiscretizeActionWrapper(env, num_bins=5)

    # Ejemplo de bucle principal con un agente aleatorio
    NUM_EPISODES = 1
    for ep in range(NUM_EPISODES):
        print(f"Episodio {ep+1}")
        obs, info = env.reset()
        done = False
        total_reward = 0.0
        while not done:
            # Acciones aleatorias discretes: [0..num_bins-1] para cada dimensión
            discrete_action = env.action_space.sample()
            print(f"Observación {obs}. Acción {discrete_action}")
            obs, reward, done, truncated, info = env.step(discrete_action)
            total_reward += reward

    env.close()


Episodio 1
Observación [4 7 4 4 4 4 0 0]. Acción [3 3]
Observación [4 7 4 4 4 4 0 0]. Acción [4 3]
Observación [4 7 4 4 4 4 0 0]. Acción [0 0]
Observación [4 7 4 4 4 4 0 0]. Acción [4 0]
Observación [4 7 4 4 4 4 0 0]. Acción [0 0]
Observación [4 7 4 4 4 4 0 0]. Acción [1 0]
Observación [4 7 4 4 4 4 0 0]. Acción [1 0]
Observación [4 7 4 4 4 4 0 0]. Acción [0 1]
Observación [4 6 4 4 4 4 0 0]. Acción [3 2]
Observación [4 6 4 4 4 4 0 0]. Acción [4 0]
Observación [4 6 4 4 4 4 0 0]. Acción [2 3]
Observación [4 6 4 4 4 4 0 0]. Acción [3 3]
Observación [4 6 4 4 4 4 0 0]. Acción [1 3]
Observación [4 6 4 4 4 4 0 0]. Acción [2 0]
Observación [4 6 4 4 4 4 0 0]. Acción [2 2]
Observación [4 6 4 4 4 4 0 0]. Acción [4 4]
Observación [4 6 4 4 4 4 0 0]. Acción [2 1]
Observación [4 6 4 4 4 4 0 0]. Acción [0 0]
Observación [4 6 4 4 4 4 0 0]. Acción [3 3]
Observación [4 6 4 4 4 4 0 0]. Acción [3 0]
Observación [4 6 4 4 4 4 0 0]. Acción [2 0]
Observación [4 6 4 4 4 4 0 0]. Acción [0 1]
Observación [4 6 4 4 

## **Resumen**
1. **`ObservationWrapper`**: se redefine el método `observation` para procesar la **observación**.
2. **`ActionWrapper`**: se redefine el método `action` (y opcionalmente `reverse_action`) para procesar la **acción** antes de enviarla al entorno.
3. **`RewardWrapper`**: se redefine el método `reward` para modificar la **recompensa**.

Estos wrappers te permiten **personalizar** la interacción entre el agente y el entorno sin tocar directamente el código fuente del mismo, lo que facilita la experimentación y la arquitectura limpia en tu proyecto de aprendizaje por refuerzo.

Como ejemplo de uso los hemos utilizado para discretizar las observaciones y acciones en un entorno continuo.